Apply word2vec on a subset of the dataset of Amazon Product Reviews

In [18]:
#installing the kaggle library
!pip install kaggle 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (3).json


{'kaggle.json': b'{"username":"akrambusiness","key":"3b00820d5658b305a9386288c07227f0"}'}

In [25]:
#Upload Your kaggle Json file
#We configure the path of kaggle.json file

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
##Importing  the dataset
!kaggle datasets download -d grikomsn/amazon-cell-phones-reviews

In [27]:
#!pip install gensim
!pip install python-Levenshtein

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 174.1/174.1 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 36.8 MB/s eta 0:00:00


### **Reading and Exploring the dataset**





the dataset we used in our notebook is a subset of Amazon reviews from the Cell Phones category, we gonna read it using pandas.
Link to the Dataset:
https://www.kaggle.com/datasets/grikomsn/amazon-cell-phones-reviews/download?datasetVersionNumber=2

In [26]:
import gensim
import pandas as pd

In [30]:
df = pd.read_csv("/content/Cell_Phones_reviews.csv",delimiter =',')
df.head()

,asin,name,rating,date,verified,title,body,helpfulVotes
0,B0000SX2UC,Janet,3,"October 11, 2005",False,"Def not best, but not worst",I had the Samsung A600 for awhile which is abs...,1.0
1,B0000SX2UC,Luke Wyatt,1,"January 7, 2004",False,Text Messaging Doesn't Work,Due to a software issue between Nokia and Spri...,17.0
2,B0000SX2UC,Brooke,5,"December 30, 2003",False,Love This Phone,"This is a great, reliable phone. I also purcha...",5.0
3,B0000SX2UC,amy m. teague,3,"March 18, 2004",False,"Love the Phone, BUT...!","I love the phone and all, because I really did...",1.0
4,B0000SX2UC,tristazbimmer,4,"August 28, 2005",False,"Great phone service and options, lousy case!",The phone has been great for every purpose it ...,1.0


We will train a word2vec model using only the column "body" as we can also use the "title", we can delete some unecessary columns or just ignore them

In [36]:
df.body[0]

"I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> Recep

In [33]:
df.shape

(67986, 7)

### **Simple Preprocessing**

this simple function in gensim libray can do some basic preprocessing operations like converting all the words to lower case, trimming spaces, removing punctuations and stop words for our first review as an example to show you 

In [37]:
gensim.utils.simple_preprocess("I had the Samsung A600 for awhile which is absolute doo doo. You can read my review on it and detect my rage at the stupid thing. It finally died on me so I used this Nokia phone I bought in a garage sale for $1. I wonder y she sold it so cheap?... Bad: ===> I hate the menu. It takes forever to get to what you want because you have to scroll endlessly. Usually phones have numbered categories so u can simply press the # and get where you want to go. ===> It's a pain to put it on silent or vibrate. If you're in class and it rings, you have to turn it off immediately. There's no fast way to silence the damn thing. Always remember to put it on silent! I learned that the hard way. ===> It's so true about the case. It's a mission to get off and will break ur nails in the process. Also, you'll damage the case each time u try. For some reason the phone started giving me problems once I did succeed in opening it. ===> Buttons could be a bit bigger. Vibration could be stronger. Good: ===> Reception is not too shabby. I was using it in the elevator which is a remarkable feat considering my old phone would lose service by simply putting it in my pocket. ===> Compared to my old Samsung, this phone works quite well. The ring tones are loud enough to hear and the phone actually charges quickly and has great battery life. It doesn't heat up like a potatoe in the oven either during long phone convos. ===> Nice bright, large screen. ===> Cute ways to customize it. Scroll bar can be set to purple, pink, aqua, orange, etc. Overall: Okay phone. It serves its purpose but definitely pales in comparison to these new phones coming out from Sprint. Why get so so when you can get great?")

['had',
 'the',
 'samsung',
 'for',
 'awhile',
 'which',
 'is',
 'absolute',
 'doo',
 'doo',
 'you',
 'can',
 'read',
 'my',
 'review',
 'on',
 'it',
 'and',
 'detect',
 'my',
 'rage',
 'at',
 'the',
 'stupid',
 'thing',
 'it',
 'finally',
 'died',
 'on',
 'me',
 'so',
 'used',
 'this',
 'nokia',
 'phone',
 'bought',
 'in',
 'garage',
 'sale',
 'for',
 'wonder',
 'she',
 'sold',
 'it',
 'so',
 'cheap',
 'bad',
 'hate',
 'the',
 'menu',
 'it',
 'takes',
 'forever',
 'to',
 'get',
 'to',
 'what',
 'you',
 'want',
 'because',
 'you',
 'have',
 'to',
 'scroll',
 'endlessly',
 'usually',
 'phones',
 'have',
 'numbered',
 'categories',
 'so',
 'can',
 'simply',
 'press',
 'the',
 'and',
 'get',
 'where',
 'you',
 'want',
 'to',
 'go',
 'it',
 'pain',
 'to',
 'put',
 'it',
 'on',
 'silent',
 'or',
 'vibrate',
 'if',
 'you',
 're',
 'in',
 'class',
 'and',
 'it',
 'rings',
 'you',
 'have',
 'to',
 'turn',
 'it',
 'off',
 'immediately',
 'there',
 'no',
 'fast',
 'way',
 'to',
 'silence',
 'the

Now we gonna apply it to the entire body column

In [39]:
review_text = df.body.astype(str).apply(gensim.utils.simple_preprocess)

In [40]:
review_text

0        [had, the, samsung, for, awhile, which, is, ab...
1        [due, to, software, issue, between, nokia, and...
2        [this, is, great, reliable, phone, also, purch...
3        [love, the, phone, and, all, because, really, ...
4        [the, phone, has, been, great, for, every, pur...
                               ...                        
67981    [love, the, camera, on, this, phone, the, scre...
67982    [ve, been, an, xperia, user, for, several, yea...
67983                    [buy, one, more, for, my, cousin]
67984    [product, looks, and, works, like, new, very, ...
67985    [love, the, size, and, style, of, this, phone,...
Name: body, Length: 67986, dtype: object

We created a pandas seriesn, each object in those series is a list of tokenized words 

###  **Training the model**

Now we gonna create our model

In [41]:
model = gensim.models.Word2Vec(window = 10, min_count = 2, workers = 4)

In [43]:
#building vocabulary
model.build_vocab(review_text, progress_per=100)

In [44]:
model.corpus_count

67986

In [45]:
#training the model
model.train(review_text, total_examples= model.corpus_count, epochs =model.epochs)

(12780938, 17372980)

  Let's evaluate our model by doing some some similarities

In [46]:
model.wv.most_similar("bad")

[('terrible', 0.6830868721008301),
 ('horrible', 0.6353497505187988),
 ('good', 0.6071088910102844),
 ('poor', 0.5469701886177063),
 ('crappy', 0.5149475932121277),
 ('disappointing', 0.5011484026908875),
 ('sucks', 0.49565616250038147),
 ('weak', 0.4690515995025635),
 ('sad', 0.4608418643474579),
 ('okay', 0.4497946798801422)]

In [48]:
#We could see similarity between two words
model.wv.similarity(w1= "cheap", w2 = "inexpensive")

0.40743205

In [49]:
model.wv.similarity(w1= "good", w2 = "great")

0.75749326

In [ ]:
model.wv.similarity(w1= "cheap", w2 = "inexpensive")

In [51]:
model.wv.similarity(w1= "great", w2 = "product")

-0.19689213